# TECHNICAL REPORT: VRP with Genetic Algorithm (Initial Problem)

## 1. Introduction and Objectives

The main objective of this experiment is to solve a simplified version of the **Vehicle Routing Problem (VRP)** with capacity constraints (**CVRP**) using a **Genetic Algorithm (GA)**.

The VRP seeks to find the set of optimal routes for a fleet of vehicles departing from and returning to a depot to serve a set of customers. The goal is to **minimize the total distance traveled** by all vehicles, ensuring that the **load capacity** of each vehicle is not exceeded on any route.

**Specific Objectives:**

1.  **Model** the VRP with capacity constraints.
2.  **Implement** a Genetic Algorithm to find the best sequence of customers.
3.  **Evaluate** the quality of the solution (route) using the *fitness* function (total distance).
4.  **Analyze** the generated routes and their efficiency.

## 2. Experiment Design and Decisions

#### 2.1. Problem Representation

* **Customers:** 5 customers are defined with their coordinates $[x, y]$.
* **Depot:** A starting and ending point with fixed coordinates.
* **Demand:** Each customer's demand is calculated in weight (kg) based on a list of products and quantities.
* **Vehicle Capacity:** A fixed maximum capacity of **$60$ kg** is established.

#### 2.2. Preprocessing: Customer Classification

A crucial step of classifying customers into **valid** and **invalid** is performed:

* **Valid Customers:** Those whose individual demand is **less than or equal** to the vehicle's capacity.
* **Invalid Customers:** Those whose individual demand is **greater** than the capacity. These customers are excluded from the routing problem.

#### 2.3. Genetic Algorithm (GA) - Design Decisions

| Component | Design Decision | Explanation |
| :--- | :--- | :--- |
| **Chromosome (Individual)** | A list (permutation) of **valid customer indices**. | The *global* route is a complete sequence of customers to visit. |
| **Fitness Function** | $\text{fitness}(\text{route}) = \text{Total Distance}(\text{route})$ | **Minimize the total distance** traveled (Euclidean distance). |
| **Crossover Operator** | **Order Crossover (OX)** modified. | A segment from parent 1 is maintained and completed with elements from parent 2, maintaining validity (permutation without repetitions). |
| **Mutation Operator** | **Swap**. | With a 10% probability, two random positions in the route are swapped. |
| **Selection Operator** | **Binary Tournament**. | Two individuals are selected, and the one with the **lower** *fitness* (shorter route) is chosen. |
| **Routing (split_into_routes)** | **Capacity-First Heuristic**. | The sequence of customers is divided into individual trips (routes) to the depot **whenever the accumulated load would exceed the vehicle's capacity**. Each trip starts and ends at the depot. |

The GA is used to optimize the **visiting sequence of valid customers**.

## 3. Experiment Execution

#### 3.1. Demand Data and Classification

| Original Customer | Calculated Total Demand (kg) | Classification |
| :---: | :---: | :---: |
| C1 | 18.6 | Valid |
| C2 | 22.8 | Valid |
| C3 | 48.0 | Valid |
| C4 | 60.0 | Valid |
| C5 | 77.7 | **Invalid** (> 60 kg) |

* **Valid Customers for GA:** 4 customers (indices 0, 1, 2, 3).

#### 3.2. Results (Example Run)

*(Note: GA results are stochastic and may vary with each execution.)*

* **Best Visit Order (Chromosome):** `[3, 0, 1, 2]` (Corresponds to sequence: C4, C1, C2, C3)
* **Best Total Distance:** $230.13$ (distance units)

#### 3.3. Route Division (According to Optimal Sequence)

The sequence `[3, 0, 1, 2]` is divided into trips respecting the $60$ kg limit:

1.  **Trip 1:** Serves customer 3 (C4). Load: 60.0 kg.
2.  **Trip 2:** Serves customers 0 and 1 (C1 and C2). Load: $41.4$ kg ($18.6 + 22.8$).
3.  **Trip 3:** Serves customer 2 (C3). Load: $48.0$ kg.

## 4. Analysis

#### 4.1. Solution Analysis

1.  **Constraint Handling:** Preprocessing that identifies Customer 5 as **Invalid** is essential, ensuring the GA only operates within a feasible solution space. The *fitness* function forces the creation of multiple trips (3 in this case) for the set of valid customers.
2.  **GA Efficiency:** By optimizing the **visit sequence**, the Genetic Algorithm manages to find an order that minimizes total distance while considering the 'interruptions' forced by capacity. It is observed that the GA prioritizes grouping low-demand customers (C1 and C2) and isolating the limit-demand customer (C4) in their own trip, which is logically efficient.
3.  **Fitness Function and Division Heuristic:** The fitness function intrinsically penalizes orders that force many long trips, as each trip implies two extra distance segments: **Depot $\rightarrow$ First Customer** and **Last Customer $\rightarrow$ Depot**. This directs the GA to search for sequences that allow multi-stop trips.

#### 4.2. Limitations

* **Simple Division Heuristic:** The `dividir_en_rutas` function uses a simple 'first-fit' strategy. In more complex problems, a smarter division that minimizes the distance of the *individual trip* could improve the solution.
* **Multi-Objective:** The solution only minimizes distance. A more complete VRP would also seek to **minimize the number of vehicles** (routes). The GA could be modified to include a penalty in the *fitness* for each additional route generated.

## 5. VRP and GA Code Implementation

Below is the code implementing the logic described in the report.

### VRP with Genetic Algorithm
Notebook organized into multiple cells for better readability.

### 5.1. Library Import
This section includes the necessary libraries to implement the genetic algorithm and auxiliary functions.

In [ ]:
import random
import math
import matplotlib.pyplot as plt

### 5.2. Problem Data
Coordinates for the depot and customers, product weights, and customer orders are established.

In [ ]:
# Depot coordinates
depot = [20, 120]

# Customer coordinates
clientes = [
    [35, 115],
    [50, 140],
    [70, 100],
    [40, 80],
    [25, 60]
]

# Product weights (kg)
pesos = [1.2, 3.8, 7.5, 0.9, 15.4, 12.1, 4.3, 19.7, 8.6, 2.5]

# Customer orders: (product, quantity)
pedidos = [
    [(3, 2), (1, 3)],
    [(2, 6)],
    [(7, 4), (5, 2)],
    [(3, 8)],
    [(6, 5), (9, 2)]
]

print('Data loaded successfully.')

### 5.3. Demand Calculation
Total demand for each customer is calculated by summing the weight of ordered products.

In [ ]:
demandas = []
for pedido in pedidos:
    total = sum(pesos[item - 1] * cant for item, cant in pedido)
    demandas.append(total)

capacidad = 60
print('Calculated demands:', [round(x,1) for x in demandas])

### 5.4. Classification of Valid / Invalid Customers (Preprocessing)

In [ ]:
clientes_validos = []
demandas_validas = []
clientes_invalidos = []
demandas_invalidas = []

for i in range(len(clientes)):
    if demandas[i] <= capacidad:
        # Save valid customer. The index corresponds to the position in the valid list.
        # For GA simplicity, we only store coordinates for distance calculation and their demands.
        clientes_validos.append(clientes[i])
        demandas_validas.append(demandas[i])
    else:
        clientes_invalidos.append(clientes[i])
        demandas_invalidas.append(demandas[i])

print('Valid customers:', len(clientes_validos))
print('Valid demands:', [round(d, 1) for d in demandas_validas])
print('Invalid customers:', len(clientes_invalidos))
print('Invalid demands:', [round(d, 1) for d in demandas_invalidas])

### 5.5. Auxiliary Functions (Fitness)
These are helper functions used by the fitness function to evaluate each solution of the genetic algorithm.

In [ ]:
def distancia(a, b):
    """Calculates Euclidean distance between two points."""
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def dividir_en_rutas(ruta):
    """Divides the global sequence (chromosome) into valid routes by capacity (First Fit Heuristic)."""
    rutas = []
    carga = 0
    actual = []
    for c in ruta:
        # c is the index of the valid customer (0 to n-1)
        if carga + demandas_validas[c] <= capacidad:
            actual.append(c)
            carga += demandas_validas[c]
        else:
            # Capacity exceeded: current route ends, new one begins.
            rutas.append(actual)
            actual = [c]
            carga = demandas_validas[c]
    if actual:
        # Add the last route if it exists
        rutas.append(actual)
    return rutas

def distancia_total(ruta):
    """Calculates total distance of all trips generated from the route (chromosome)."""
    rutas = dividir_en_rutas(ruta)
    total = 0
    for r in rutas:
        pos = depot # Start at depot
        for c in r:
            # Distance from previous point to customer c
            total += distancia(pos, clientes_validos[c])
            pos = clientes_validos[c]
        # Return distance to depot
        total += distancia(pos, depot)
    return total

def fitness(ruta):
    """Fitness is the total distance (we seek to minimize it)."""
    return distancia_total(ruta)

print('Auxiliary functions (distance, route division, and fitness) defined.')

### 5.6. Genetic Operators
Genetic operators define how new solutions are created in each generation:

- Creation: creates an initial population of n random routes.

- Selection: chooses best individuals with lower fitness.

- Crossover: combines two parents to generate offspring maintaining customer order (OX).

- Mutation: introduces small variations by swapping.

In [ ]:
def crear_poblacion(n, n_clientes):
    """Creates an initial population of n random routes."""
    poblacion = []
    base = list(range(n_clientes))
    for _ in range(n):
        r = base[:]
        random.shuffle(r)
        poblacion.append(r)
    return poblacion

def seleccion(poblacion, fitnesses):
    """Selection by binary tournament (chooses the individual with lower fitness)."""
    i1, i2 = random.sample(range(len(poblacion)), 2)
    return poblacion[i1][:] if fitnesses[i1] < fitnesses[i2] else poblacion[i2][:]

def cruce(p1, p2):
    """Order Crossover (OX) modified to maintain chromosome validity."""
    a, b = sorted(random.sample(range(len(p1)), 2))
    hijo = [-1]*len(p1)
    hijo[a:b] = p1[a:b]
    pos = b
    for x in p2:
        if x not in hijo:
            if pos >= len(p1): pos = 0
            hijo[pos] = x
            pos += 1
    return hijo

def mutacion(ruta, prob=0.1):
    """Mutation by swap with a given probability."""
    if random.random() < prob:
        i, j = random.sample(range(len(ruta)), 2)
        ruta[i], ruta[j] = ruta[j], ruta[i]

print('Genetic operators ready.')

### 5.7. Main Genetic Algorithm
This is the core of the GA that seeks, generation after generation, to minimize the total distance of routes while respecting vehicle capacity.

In [ ]:
def algoritmo_genetico(n_generaciones=200, tam_pob=50):
    """Main loop of the Genetic Algorithm."""
    n_clientes = len(clientes_validos)
    if n_clientes == 0:
        return [], 0
        
    poblacion = crear_poblacion(tam_pob, n_clientes)
    fitnesses = [fitness(r) for r in poblacion]

    for gen in range(n_generaciones):
        nueva = []
        for _ in range(tam_pob):
            p1 = seleccion(poblacion, fitnesses)
            p2 = seleccion(poblacion, fitnesses)
            hijo = cruce(p1, p2)
            mutacion(hijo, 0.1)
            nueva.append(hijo)
        poblacion = nueva
        fitnesses = [fitness(r) for r in poblacion]

    mejor = min(range(tam_pob), key=lambda i: fitnesses[i])
    return poblacion[mejor], fitnesses[mejor]

print('Genetic algorithm configured.')

### 5.8. Execution and Results

In this section, the algorithm results are displayed.

In [ ]:
mejor_ruta, mejor_valor = algoritmo_genetico()

print("Visit Order (Indices of valid customers):")
print(mejor_ruta)
print("Total Distance:", round(mejor_valor, 2))

print("\nTrip Details:")
rutas = dividir_en_rutas(mejor_ruta)
for i, r in enumerate(rutas):
    carga = sum(demandas_validas[c] for c in r)
    print(f"Trip {i+1}: {r} (Load {round(carga,1)} kg)")

### 5.9. Final Visualization

This section shows a plot with the routes, the depot, weights, and customers.

In [ ]:
def plot_rutas(rutas):
    """Generates the plot of routes, depot, and customers."""
    plt.figure(figsize=(9, 9))
    
    # Valid Customers
    for i, (x, y) in enumerate(clientes_validos):
        plt.scatter(x, y, c='blue', s=80, zorder=5)
        plt.text(x+1, y+1, f'C{i+1} ({round(demandas_validas[i], 1)} kg)', fontsize=9, color='blue', fontweight='bold')
        
    # Depot
    plt.scatter(depot[0], depot[1], c='red', s=150, marker='s', label='Depot', zorder=6)

    colores = ['green', 'orange', 'purple', 'cyan', 'brown']

    # Route Plotting
    for i, r in enumerate(rutas):
        puntos = [depot] + [clientes_validos[c] for c in r] + [depot]
        xs = [p[0] for p in puntos]
        ys = [p[1] for p in puntos]
        plt.plot(xs, ys, '-o', color=colores[i % len(colores)], alpha=0.6, label=f'Trip {i+1}')
        carga = sum(demandas_validas[c] for c in r)
        
    # Invalid Customers (Excluded)
    for (x, y) in clientes_invalidos:
        idx = clientes.index([x, y]) # Find original index to get demand
        demanda_inv = demandas[idx]
        plt.scatter(x, y, c='gray', marker='x', s=100, zorder=4)
        plt.text(x-10, y-10, f'Invalid: {round(demanda_inv, 1)} kg', fontsize=9, color='gray')

    plt.title("Optimal Vehicle Routes (Capacity 60 kg)")
    plt.xlabel("X Coordinate")
    plt.ylabel("Y Coordinate")
    plt.legend()
    plt.grid(True)
    plt.show()

if mejor_ruta:
    plot_rutas(rutas)
else:
    print("No valid customers to plot.")

## 6. Final Experiment Conclusions

After executing the genetic algorithm with a capacity constraint of **60 kg**, the following conclusions are drawn:

1.  **Demand Loss (Invalid Customer):**
    * The system correctly identified that **Customer 5** has a demand of **77.7 kg**, which exceeds the maximum vehicle capacity.
    * **Conclusion:** It is impossible to satisfy this customer's demand with the current fleet. This represents a significant service loss and suggests the need for larger vehicles or allowing partial deliveries (splitting the order into two trips).

2.  **Fleet Efficiency (Underutilization):**
    * To serve the remaining 4 valid customers, the algorithm generated **3 trips**.
    * *Trip with C4 (60 kg):* 100% occupancy. Very efficient.
    * *Trip with C3 (48 kg):* 80% occupancy. With only 12 kg free, C1 (18.6 kg) or C2 (22.8 kg) could not be added, forcing a new trip.
    * **Conclusion:** The 60 kg constraint causes high route fragmentation. Many return trips to the depot are required, increasing total distance and operating costs.
    
3.  **Genetic Algorithm Performance:**
    * The GA managed to find a permutation that minimizes distance given the constraints, effectively grouping small customers (C1 and C2) where possible.
    * However, the quality of the logistical solution is strongly limited by the physical capacity of the vehicle, not by the algorithm's optimization.